<a href="https://colab.research.google.com/github/kjm12misa/Google_drive/blob/main/Python/chapter_5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 교차 검증과 그리드 서치
- 키워드 : 하이퍼 파라미터(그리드 서치 vs 랜덤서치)
- 데이터가 작을 때 주로 사용
- 하이퍼파라미터
  + max_depth : 3, 정확도 84%
- 결론
  + 모른다면 디폴트만 쓰자 또는 다 써보고 머가 좋은지 고른다.
  + 가성비(시간대비 성능 보장이 안됨)

#### 검증 세트
- 테스트 세트 (1회성)
- 훈련 데이터를 훈련데이터 + 검증 데이터로 재 분할

#### 현실
- 테스트 데이터가 별도로 존재하지 않고, 항상 데이터가 존재하지는 않기 때문에 그 전날의 데이터를 모아 테스트 데이터를 분류하여 이전 버전과 비교 후 더 좋은거 사용
- 전체 데이터 - 훈련(6) : 검증(2) : 테스트(2) --> 보통 관례적으로 이렇게 구분함.
  + 훈련-검증-테스트 결과 안좋으면 다시 훈련-검증,,훈련-검증,,,
  + 테스트 데이터는 모르는 데이터로 생각?

#### 데이터 불러오기

In [ ]:
import pandas as pd
wine = pd.read_csv('http://bit.ly/wine_csv_data')

In [ ]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target= wine['class'].to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, random_state=42)

sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size = 0.2, random_state=42)
print(sub_input.shape, val_input.shape)

(3897, 3) (975, 3)


In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.997947138824737
0.8420512820512821


#### 교차 검증의 목적
- 좋은 모델이 만들어짐.
  + 좋은 모델 != 성능 좋은 모델
  + 좋은 모델 = 과대적합이 아니고 모형의 오차가 적고 안정적인 모델
- P.245 <시간대비 가성비가 중요>
  + 모델평가 1 : 90% (소요시간 1시간)
  + 모델평가 2 : 85%
  + 모델평가 3 : 80%
- 단점 : 데이터의 양이 많으면 시간이 많이 소요됨. 적은 양에 더 잘 쓰임.

#### 교차 검증 함수↓

In [ ]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00883842, 0.0072813 , 0.00674725, 0.00664949, 0.00726604]), 'score_time': array([0.00073314, 0.00057125, 0.00085616, 0.0005827 , 0.00072289]), 'test_score': array([0.85128205, 0.84820513, 0.8788501 , 0.85112936, 0.84394251])}


#### 최정 점수 평균 구하기

In [ ]:
import numpy as np
print(np.mean(scores['test_score']))

0.8546818301479492


- 훈련 세트 섞은 후 10-폴드 교차 검증

In [ ]:
from sklearn.model_selection import StratifiedKFold
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv = splitter)

print(np.mean(scores['test_score']))

0.868634917696166


#### 하이퍼 파라미터 튜닝을 할 시에는 랜덤 서치 사용.
- 요즘은 하이퍼 파라미터를 자동으로 잡아주는 라이브러리가 존재
  + hyperopt

- n_jobs 의미(작업수를 늘리면 시간 감소)
  + 현재 내가 가지고 있는 모든 something,,을 써서 하겠다.
  + n_module은 동시에 실행 중인 워커의 최대 수를 지정하는 정수이다. 1을 지정하면 joblib 병렬화가 전혀 사용되지 않으므로 디버깅에 도움이 됩니다. -1로 설정하면 모든 CPU가 사용됩니다. -1 미만의 n_jobs의 경우 (n_cpus + 1 + n_jobs)가 사용됩니다.
    + 출처 : 구글 검색(n_jobs = -1/People also ask)

In [ ]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs = -1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

- gridsearchcv best estimator
  + 누락된 데이터에 대해 가장 높은 점수(또는 지정된 경우 가장 작은 손실)를 준 추정기
  + refit=False이면 사용할 수 없다.

In [ ]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))
print(gs.best_params_)
print(gs.cv_results_['mean_test_score'])

0.9137931034482759
{'min_impurity_decrease': 0.0003}
[0.86843111 0.86925267 0.87315179 0.87212531 0.87130627]


#### 랜덤 서치 P.252
- 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 있도록 확률 분포 객체를 전달.(범위를 잡아준 후 나름의 확률에 인해서,,)

In [ ]:
from scipy.stats import uniform, randint
rgen = randint(0, 10)
rgen.rvs(10)

array([2, 8, 8, 9, 2, 9, 5, 1, 5, 9])

In [ ]:
np.unique(rgen.rvs(1000), return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([107,  96,  90,  98,  98,  95, 108,  97, 112,  99]))

- randint()
  + 랜덤 정수 생성(포함 범위로 부터 임의의 정수값)
  + eg. randint(0. 5) -> [0, 1, 2, 3, 4, 5]의 난수를 반환한다.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
params = {
    'min_impurity_decrease': uniform(0.0001, 0.001),
    'max_depth': randint(20, 50)}

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42),
                        params, n_iter=100, n_jobs = -1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fa8927441d0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fa89273bf10>},
                   random_state=42)

In [ ]:
gs.best_params_

{'max_depth': 21, 'min_impurity_decrease': 0.0002818249672071006}